# Fundamental Groups and Orbifolds of Parabolic Arrangement Complements

## A worked example: type $A_3$, $k = 3$

This notebook accompanies the paper and demonstrates the end-to-end computation of $\pi_1(K_{\mathscr{A}})$ using the Bridson-Haefliger complex-of-groups machinery.

---

**The central exact sequence:**
$$1 \;\longrightarrow\; \pi_1(K_{\mathscr{A}}) \;\longrightarrow\; \mathcal{G}_{\mathscr{A}} \;\xrightarrow{\;\rho\;} \; W \;\longrightarrow\; 1$$

**Structure of this notebook:**

### Part A — The $W$-invariant case
1. The restricted permutahedron $K_{\mathscr{A}}$ and its cell structure
2. Orbits and local groups under the $W$-action
3. The quotient scwol $\mathcal{Y}$ and spanning tree $T$
4. Morphisms $\psi_a$ and the torsion element
5. The orbifold group $\mathcal{G}_{\mathscr{A}}$ (modified Coxeter group)
6. Computation of $\pi_1 = \ker(\rho)$ as $F_7$
7. 3D visualization of a generator in the permutahedron

### Part B — The $H$-invariant case and cross-validation
8. Choice of $H = \langle s_1, s_2 \rangle$ and $H$-invariance
9. Transport and twisting elements (canonical and non-canonical sections)
10. Three-route verification: W-route, full B&H, and H-route

## 1. Load Module and Build Arrangement

In [ ]:
load('../parabolic_arrangements.sage')

# A_3, k=3 arrangement
W, Plist, _ = build_W_P('A', 3)
Delta = ideal_k_parabolic(W, Plist, k=3)
dga = ParabolicArrangementCohomology(W, Plist, Delta)

S = dga.S
sref = dga._sref
print(f'W = {W.cartan_type()},  |W| = {W.cardinality()}')
print(f'Generators S = {S}')
print(f'Excluded cells: {len(Delta)},  remaining: {len(dga.cells)}')

---
## Part A: The $W$-Invariant Case

## 2. The Restricted Permutahedron $K_{\mathscr{A}}$

The permutahedron of $A_3$ is the **truncated octahedron**: 24 vertices, 36 edges, 14 faces
(8 hexagonal + 6 square). The **Coxeter matrix** determines the face types.

The $k=3$ arrangement removes the hexagonal faces (stratum $\sigma_{s,t}$ with $m_{st}=3$,
adjacent generators). Only the 6 square faces remain ($\sigma_{1,3}$, $m_{13}=2$).

In [ ]:
# Coxeter matrix
print('Coxeter matrix m_{st}:')
for i in range(len(S)):
    for j in range(i+1, len(S)):
        s, t = S[i], S[j]
        m = (sref[s] * sref[t]).order()
        print(f'  m_{{{s},{t}}} = {m}')

print()
print('Cells by dimension in K_A:')
for k in range(dga.max_grade + 1):
    cells_k = dga.by_grade.get(k, [])
    print(f'  dim {k}: {len(cells_k)} cells', end='')
    if k == 0: print('  (vertices = elements of W)')
    elif k == 1: print('  (edges = rank-1 cosets)')
    elif k == 2:
        sq = sum(1 for c in cells_k if len(c[1])==2)
        print(f'  (2-faces: {sq} squares retained, hexagons REMOVED)')
    else: print()

# Euler characteristic verification
chi = sum((-1)**k * len(dga.by_grade.get(k,[])) for k in range(dga.max_grade+1))
print(f'\nEuler characteristic chi(K_A) = {chi}')
print('Check: chi = 1 - beta_1 + beta_2 = 1 - 7 + 0 =', 1 - 7 + 0)

### Interactive Visualization of $K_{\mathscr{A}}$

The complex $K_{\mathscr{A}}$ lives as a subspace of the **truncated octahedron** (permutahedron of $A_3$):
- Edges coloured by simple generator $s_1$ (red), $s_2$ (green), $s_3$ (blue)
- **Blue faces** = 6 retained squares (type $\{s_1, s_3\}$, non-adjacent pairs)
- **Grey faces** = 8 removed hexagons (what the arrangement $\mathscr{A}$ eliminates)

In [ ]:
import numpy as np
import plotly.graph_objects as go
from itertools import permutations as iperms
from collections import defaultdict
from sage.geometry.polyhedron.constructor import Polyhedron

# ── Geometry ──────────────────────────────────────────────────────────────────
all_verts = sorted(set(iperms([1,2,3,4])))
center_4d = np.mean(np.array(all_verts, dtype=float), axis=0)

c1 = np.array([ 1,-1,-1, 1], dtype=float) / 2
c2 = np.array([ 1, 1,-1,-1], dtype=float) / 2
c3 = np.array([ 1,-1, 1,-1], dtype=float) / 2

def proj(v):
    cv = np.array(v, dtype=float) - center_4d
    return np.array([cv @ c1, cv @ c2, cv @ c3])

coords = {v: proj(v) for v in all_verts}
poly_center = np.mean(np.array([coords[v] for v in all_verts]), axis=0)

# ── Edge type classification (pos vs value swaps) ─────────────────────────────
# pos-swap: swap adjacent POSITIONS i,i+1 in the one-line notation.
# val-swap: swap adjacent VALUES i,i+1 (i.e., left multiplication).

def edge_type_pos(u, v):
    diff = [i for i in range(4) if u[i] != v[i]]
    if len(diff) == 2 and abs(diff[0]-diff[1]) == 1:
        if u[diff[0]] == v[diff[1]] and u[diff[1]] == v[diff[0]]:
            return diff[0] + 1
    return None


def edge_type_val(u, v):
    for i in (1, 2, 3):
        ulist = list(u)
        pi = ulist.index(i)
        pj = ulist.index(i + 1)
        ulist[pi], ulist[pj] = ulist[pj], ulist[pi]
        if tuple(ulist) == v:
            return i
    return None

# Use Sage 1-faces to guarantee the 1-skeleton is correct
P        = Polyhedron(vertices=all_verts)
sage_hex = [f for f in P.faces(2) if f.n_vertices() == 6]
sage_sq  = [f for f in P.faces(2) if f.n_vertices() == 4]

edge_pairs = []
for f in P.faces(1):
    vts = [tuple(int(x) for x in v.vector()) for v in f.vertices()]
    assert len(vts) == 2
    edge_pairs.append((vts[0], vts[1]))


def classify_edges(edge_pairs, edge_type_fn):
    counts = {1: 0, 2: 0, 3: 0, None: 0}
    typed = []
    for u, v in edge_pairs:
        t = edge_type_fn(u, v)
        counts[t] = counts.get(t, 0) + 1
        typed.append((u, v, t))
    return typed, counts

edge_pos, counts_pos = classify_edges(edge_pairs, edge_type_pos)
edge_val, counts_val = classify_edges(edge_pairs, edge_type_val)

if counts_pos.get(None, 0) == 0:
    edge_list = edge_pos
    edge_mode = 'posiciones'
elif counts_val.get(None, 0) == 0:
    edge_list = edge_val
    edge_mode = 'valores'
else:
    edge_list = edge_pos
    edge_mode = 'posiciones (con None)'

print('Edges total:', len(edge_pairs))
print('Counts pos-swap:', counts_pos)
print('Counts val-swap:', counts_val)
print('Usando modo:', edge_mode)

# ── Face ordering + normals (for outward offset of lines) ─────────────────────

def ordered_face(f):
    vt  = [tuple(int(x) for x in v.vector()) for v in f.vertices()]
    pts = np.array([coords[v] for v in vt])
    cen = pts.mean(axis=0)
    e1  = pts[1] - pts[0]; e1 /= np.linalg.norm(e1)
    nrm = np.cross(pts[1]-pts[0], pts[2]-pts[0]); nrm /= np.linalg.norm(nrm)
    e2  = np.cross(nrm, e1)
    order = np.argsort(np.arctan2([(p-cen)@e2 for p in pts],
                                 [(p-cen)@e1 for p in pts]))
    vt_ord  = [vt[i] for i in order]
    pts_ord = pts[order]
    nrm = np.cross(pts_ord[1]-pts_ord[0], pts_ord[2]-pts_ord[0])
    nrm = nrm / np.linalg.norm(nrm)
    if np.dot(nrm, pts_ord.mean(axis=0) - poly_center) < 0:
        nrm = -nrm
    return vt_ord, pts_ord, nrm

faces_by_edge = defaultdict(list)
for f in P.faces(2):
    vt_ord, _, nrm = ordered_face(f)
    m = len(vt_ord)
    for i in range(m):
        a = vt_ord[i]
        b = vt_ord[(i+1) % m]
        faces_by_edge[frozenset([a, b])].append(nrm)

# ── Face triangulation ───────────────────────────────────────────────────────

def ordered_face_pts(f):
    _, pts, _ = ordered_face(f)
    return pts


def _triangulate(face_list):
    vx, vy, vz, fi, fj, fk = [], [], [], [], [], []
    for f in face_list:
        pts = ordered_face_pts(f)
        n = len(pts); base = len(vx)
        for p in pts:
            vx.append(float(p[0])); vy.append(float(p[1])); vz.append(float(p[2]))
        for t in range(1, n-1):
            fi.append(base); fj.append(base+t); fk.append(base+t+1)
    return vx, vy, vz, fi, fj, fk


def clean_scene(eye=(2.5, 0.3, 0.3)):
    ax = dict(visible=False)
    return dict(xaxis=ax, yaxis=ax, zaxis=ax,
                bgcolor='white', aspectmode='cube',
                camera=dict(eye=dict(x=eye[0], y=eye[1], z=eye[2])))

# ── Permutaedro completo: todas las caras (sólido cerrado) ───────────────────
_flat = dict(ambient=1.0, diffuse=0.0, specular=0.0, roughness=1.0, fresnel=0.0)
FACE_OPACITY = 0.65  # ajusta si quieres más/menos transparencia
EDGE_OFFSET  = 0.02  # pequeño empuje para evitar z-fighting

traces_perm = []

# Square faces — blue
sq_vx, sq_vy, sq_vz, sq_fi, sq_fj, sq_fk = _triangulate(sage_sq)
traces_perm.append(go.Mesh3d(
    x=sq_vx, y=sq_vy, z=sq_vz, i=sq_fi, j=sq_fj, k=sq_fk,
    color='#3182bd', opacity=FACE_OPACITY, lighting=_flat,
    name=f'{len(sage_sq)} cuadrados'))

# Hexagonal faces — light gray
hx_vx, hx_vy, hx_vz, hx_fi, hx_fj, hx_fk = _triangulate(sage_hex)
traces_perm.append(go.Mesh3d(
    x=hx_vx, y=hx_vy, z=hx_vz, i=hx_fi, j=hx_fj, k=hx_fk,
    color='#d9d9d9', opacity=FACE_OPACITY, lighting=_flat,
    name=f'{len(sage_hex)} hexágonos'))

# ── 1-skeleton as lines (Scatter3d) with outward offset ──────────────────────
# Draw all edges (no missing). Colors follow chosen edge_mode classification.

ec = {1: '#d62728', 2: '#2ca02c', 3: '#1f77b4'}
en = {1: 's₁-aristas', 2: 's₂-aristas', 3: 's₃-aristas'}

# If any edge has t=None, draw it in black so nothing is missing.
extra_x, extra_y, extra_z = [], [], []

for et in [1, 2, 3]:
    xs, ys, zs = [], [], []
    for u, v, t in edge_list:
        if t != et:
            continue
        nlist = faces_by_edge.get(frozenset([u, v]), [])
        if nlist:
            nsum = np.sum(nlist, axis=0)
            nsum = nsum / np.linalg.norm(nsum)
            off = EDGE_OFFSET * nsum
        else:
            off = np.array([0.0, 0.0, 0.0])
        pu, pv = coords[u] + off, coords[v] + off
        xs += [float(pu[0]), float(pv[0]), None]
        ys += [float(pu[1]), float(pv[1]), None]
        zs += [float(pu[2]), float(pv[2]), None]
    traces_perm.append(go.Scatter3d(x=xs, y=ys, z=zs, mode='lines',
        line=dict(color=ec[et], width=5), name=en[et]))

for u, v, t in edge_list:
    if t is not None:
        continue
    nlist = faces_by_edge.get(frozenset([u, v]), [])
    if nlist:
        nsum = np.sum(nlist, axis=0)
        nsum = nsum / np.linalg.norm(nsum)
        off = EDGE_OFFSET * nsum
    else:
        off = np.array([0.0, 0.0, 0.0])
    pu, pv = coords[u] + off, coords[v] + off
    extra_x += [float(pu[0]), float(pv[0]), None]
    extra_y += [float(pu[1]), float(pv[1]), None]
    extra_z += [float(pu[2]), float(pv[2]), None]

if extra_x:
    traces_perm.append(go.Scatter3d(
        x=extra_x, y=extra_y, z=extra_z, mode='lines',
        line=dict(color='#111111', width=5), name='aristas sin tipo'))

allp = np.array([coords[v] for v in all_verts])
traces_perm.append(go.Scatter3d(
    x=allp[:,0].tolist(), y=allp[:,1].tolist(), z=allp[:,2].tolist(),
    mode='markers', marker=dict(size=4, color='#444', opacity=0.9),
    text=[str(list(v)) for v in all_verts], hoverinfo='text', name='w ∈ W = S₄'))

fig_perm = go.Figure(data=traces_perm)
fig_perm.update_layout(
    title=dict(
        text=('Permutaedro de A₃ (octaedro truncado): 6 cuadrados + 8 hexágonos<br>'
              f'<sup>colores por sᵢ (modo {edge_mode})</sup>'),
        font=dict(size=12)),
    scene=clean_scene(),
    legend=dict(x=0.01, y=0.99, bgcolor='rgba(255,255,255,0.85)', font=dict(size=10)),
    margin=dict(l=0, r=0, b=0, t=80), width=820, height=600)

fig_perm.write_html('a3_permutaedro.html')
print(f'Permutaedro: {len(all_verts)} verts  |  {len(edge_pairs)} aristas  |  '
      f'{len(sage_sq)} cuadrados  |  {len(sage_hex)} hexágonos')
fig_perm.show()



---
## 3. Orbits and Local Groups under the $W$-Action

$W$ acts on $K_{\mathscr{A}}$ by left translation: $w \cdot (uW_J) = (wu)W_J$.
The **stabilizer** of a cell $wW_J$ is $\mathrm{Stab}_W(wW_J) = wW_Jw^{-1}$.

| Dimension | Type $J$ | Orbit rep. | Stabilizer | Order |
|-----------|----------|------------|------------|-------|
| 0 | $\emptyset$ | $eW_\emptyset = \{e\}$ | $\{e\}$ | 1 |
| 1 | $\{1\}$ | $eW_{\{1\}}$ | $\langle s_1\rangle \cong \mathbb{Z}/2$ | 2 |
| 1 | $\{2\}$ | $eW_{\{2\}}$ | $\langle s_2\rangle \cong \mathbb{Z}/2$ | 2 |
| 1 | $\{3\}$ | $eW_{\{3\}}$ | $\langle s_3\rangle \cong \mathbb{Z}/2$ | 2 |
| 2 | $\{1,3\}$ (retained) | $eW_{\{1,3\}}$ | $W_{\{1,3\}} \cong \mathbb{Z}/2 \times \mathbb{Z}/2$ | 4 |

In [ ]:
# Build complex of groups with H = W (full W-action)
pi1_calc = ParabolicArrangementPi1(dga, H_gens=list(W.gens()))

# Group orbits by (dimension, type J)
from collections import defaultdict
orbit_by_type = defaultdict(list)
for oid, rep in pi1_calc.reps.items():
    _, J = rep
    orbit_by_type[J].append(oid)

print('Orbits by parabolic type J:')
print(f'{"Type J":<12} {"# orbits":>9} {"|orbit|":>8} {"|Stab|":>8}')
print('-' * 42)
for J in sorted(orbit_by_type.keys(), key=lambda j: (len(j), j)):
    oids = orbit_by_type[J]
    sample = oids[0]
    stab_size = len(pi1_calc.stabilizers[sample])
    orbit_size = W.cardinality() // stab_size
    print(f'{str(J):<12} {len(oids):>9} {orbit_size:>8} {stab_size:>8}')

In [ ]:
e = W.one()   # identity element — used throughout the notebook

# For H = W, each orbit has a unique parabolic type J.
# The stabilizer of the canonical rep (e, J) is exactly W_J = <s_j | j in J>.
# We display this in generator notation, avoiding matrix printing.

print('Representantes canónicos y sus estabilizadores:\n')
print(f'  {"Celda":<18}  {"Estabilizador = Grupo local G_v":<35}  {"Orden":>5}')
print('  ' + '-'*62)

for J in sorted(set(J for _, J in pi1_calc.reps.values()), key=lambda j: (len(j), j)):
    oid = next(oid for oid, (_, Jrep) in pi1_calc.reps.items() if Jrep == J)
    stab_order = len(pi1_calc.stabilizers[oid])

    Jlbl     = '{' + ', '.join(str(j) for j in J) + '}' if J else '∅'
    cell_str = f'e · W_{Jlbl}'

    if not J:
        gen_str = '{e}  (grupo trivial)'
    elif len(J) == 1:
        gen_str = f'⟨s_{J[0]}⟩  ≅  ℤ/2'
    else:
        gens   = ', '.join(f's_{j}' for j in sorted(J))
        orders = [(sref[J[a]]*sref[J[b]]).order()
                  for a in range(len(J)) for b in range(a+1, len(J))]
        if all(m == 2 for m in orders):
            gen_str = f'⟨{gens}⟩  ≅  ' + ' × '.join(['ℤ/2']*len(J))
        else:
            gen_str = f'⟨{gens}⟩  (m = {", ".join(str(m) for m in orders)})'

    print(f'  {cell_str:<18}  {gen_str:<35}  {stab_order:>5}')

print()
print('Nota: eW_{1,3} tiene G_f = ⟨s_1, s_3⟩ ≅ ℤ/2×ℤ/2  porque m_{1,3}=2.')
print('Los hexágonos removidos habrían tenido G = ⟨s_1,s_2⟩ ≅ S_3, pero')
print('al no existir esas celdas en K_A, sus grupos no entran en G_A.')


---
## 2.5. El Scwol Cuociente $Y = W \backslash \mathcal{X}_{\mathscr{A}}$ y el Árbol Maximal $T$

El **scwol cuociente** $Y$ es el diagrama de Hasse del poset de órbitas de $K_{\mathscr{A}}$
bajo la acción de $W$. Sus objetos y morfismos son:

| Dimensión | Objeto (órbita) | Grupo local $G_v$ |
|-----------|-----------------|-------------------|
| 0 | $eW_\emptyset$ | $\{e\}$ |
| 1 | $eW_{\{1\}}$ | $\langle s_1 \rangle$ |
| 1 | $eW_{\{2\}}$ | $\langle s_2 \rangle$ |
| 1 | $eW_{\{3\}}$ | $\langle s_3 \rangle$ |
| 2 | $eW_{\{1,3\}}$ | $\langle s_1, s_3 \rangle \cong \mathbb{Z}/2 \times \mathbb{Z}/2$ |

Los **morfismos** de $Y$ son las inclusiones de caras (de dimensión $k$ hacia $k+1$).

### Por qué necesitamos el árbol maximal $T$

La teoría de B&H exige fijar un **árbol maximal $T$ en $Y$** para:

1. **Definir elementos de transporte $h_a \in H$**: para cada morfismo $a: \sigma \to \tau$ en $Y$,
   $h_a$ es el único elemento de $H$ tal que $h_a \cdot \tilde{\sigma} = \widetilde{\text{target}}$,
   donde $\tilde{\sigma}$ es el representante canónico de $\sigma$.

2. **Definir los morfismos $\psi_a: G_\sigma \to G_\tau$** (morfismos de grupos locales):
   $$\psi_a(g) = h_a \cdot g \cdot h_a^{-1}$$
   Para $h_a = e$ (identidad), $\psi_a$ es simplemente la **inclusión** $G_\sigma \hookrightarrow G_\tau$.

3. **Determinar los generadores** del grupo fundamental: las aristas de $Y$ **fuera** del árbol $T$
   contribuyen generadores libres; las aristas de $T$ están "colapsadas" por $\psi_a = \mathrm{id}$.

In [ ]:
import plotly.graph_objects as go

# ── 2D Hasse diagram of Y ─────────────────────────────────────────────────────
J_to_pos = {
    ():     (2.0, 0.0),
    (1,):   (0.5, 1.0),
    (2,):   (2.0, 1.0),
    (3,):   (3.5, 1.0),
    (1, 3): (2.0, 2.0),
}

oid_to_J   = {oid: J for oid, (_, J) in pi1_calc.reps.items()}
oid_to_pos = {oid: J_to_pos[J] for oid, J in oid_to_J.items()}

# Spanning tree T of Y
undirected_Y   = pi1_calc.Y.to_undirected()
tree_edge_list = undirected_Y.min_spanning_tree(algorithm='Kruskal')
tree_upairs    = {frozenset([u, v]) for u, v, _ in tree_edge_list}

all_Y_edges = list(pi1_calc.Y.edges())
n_nontree   = sum(1 for u, v, _ in all_Y_edges if frozenset([u, v]) not in tree_upairs)

# Edge segments (explicit lines so they always show)
PAD = float(0.28)
tr_x, tr_y = [], []
nt_x, nt_y = [], []

# Arrow annotations (optional; lines will already show)
annotations = []
for u, v, label in all_Y_edges:
    x0, y0 = oid_to_pos[u]; x1, y1 = oid_to_pos[v]
    in_tree = frozenset([u, v]) in tree_upairs
    color   = '#2ca02c' if in_tree else '#d62728'
    width   = float(3.0) if in_tree else float(2.0)
    dx, dy  = x1-x0, y1-y0
    nrm     = (dx**2+dy**2)**0.5 or 1.0
    x0p, y0p = x0+PAD*dx/nrm, y0+PAD*dy/nrm
    x1p, y1p = x1-PAD*dx/nrm, y1-PAD*dy/nrm

    if in_tree:
        tr_x += [float(x0p), float(x1p), None]
        tr_y += [float(y0p), float(y1p), None]
    else:
        nt_x += [float(x0p), float(x1p), None]
        nt_y += [float(y0p), float(y1p), None]

    annotations.append(dict(
        ax=float(x0p), ay=float(y0p),
        x =float(x1p), y =float(y1p),
        xref='x', yref='y', axref='x', ayref='y',
        showarrow=True, arrowhead=int(2), arrowsize=float(1.3),
        arrowwidth=float(width), arrowcolor=color, opacity=0.9))

# Node labels (show generator names, not matrices)
nx_, ny_, txt_ = [], [], []
for oid in sorted(pi1_calc.reps):
    J = oid_to_J[oid]
    x, y = oid_to_pos[oid]
    nx_.append(float(x)); ny_.append(float(y))
    Jlbl = '{' + ','.join(f's{j}' for j in J) + '}' if J else '∅'
    stab_order = len(pi1_calc.stabilizers[oid])
    txt_.append(f'eW<sub>{Jlbl}</sub><br>G={stab_order}')

edge_tree = go.Scatter(x=tr_x, y=tr_y, mode='lines',
    line=dict(color='#2ca02c', width=float(3)), name='Árbol T  (h_a = e → ψ_a = inclusión)')
edge_nt = go.Scatter(x=nt_x, y=nt_y, mode='lines',
    line=dict(color='#d62728', width=float(2)), name=f'{n_nontree} morfismo(s) no-árbol')

node_tr  = go.Scatter(x=nx_, y=ny_, mode='markers+text',
    marker=dict(size=int(55), color='#aec7e8', line=dict(color='#1f77b4', width=int(2))),
    text=txt_, textposition='middle center', textfont=dict(size=int(10)),
    hoverinfo='skip', showlegend=False)

fig_Y = go.Figure(data=[edge_tree, edge_nt, node_tr])
fig_Y.update_layout(
    title=dict(
        text=('Scwol Y = W\X<sub>𝒜</sub>  —  Diagrama de Hasse<br>'
              '<sup>Flechas = inclusiones de caras. '
              'Árbol T (verde): determina ψ_a = inclusión. '
              'Aristas fuera de T (rojo): generan libres.</sup>'),
        font=dict(size=int(12))),
    annotations=annotations,
    xaxis=dict(range=[float(-0.8), float(4.8)], showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(range=[float(-0.6), float(2.8)], showgrid=False, zeroline=False, showticklabels=False),
    plot_bgcolor='#f8f9fa', width=int(640), height=int(450),
    margin=dict(l=int(20), r=int(20), t=int(90), b=int(20)),
    legend=dict(x=float(0.01), y=float(0.01), bgcolor='rgba(255,255,255,0.85)'))

for dim_y, lbl in [(0,'dim 0'), (1,'dim 1'), (2,'dim 2')]:
    fig_Y.add_annotation(x=float(-0.6), y=float(dim_y), text=lbl, showarrow=False,
        xref='x', yref='y', font=dict(size=int(11), color='#555'), xanchor='right')

fig_Y.write_html('a3_scwol_Y.html')
print(f'Y: {pi1_calc.Y.num_verts()} objetos  |  {pi1_calc.Y.num_edges()} morfismos  |  '
      f'{len(tree_edge_list)} en árbol T  |  {n_nontree} fuera de T')
fig_Y.show()



In [ ]:
# ── Transport elements h_a for each morphism in Y ────────────────────────────
# h_a is the unique h in H s.t. h · (canonical source-target cell) = canonical rep of target orbit.
# For H = W and all canonical reps = (e, J), h_a = e for every morphism.

print('Elementos de transporte h_a  (clave para definir ψ_a):\n')
print(f'  {"Morfismo en Y":^32}  {"En árbol T":^10}  {"h_a":^20}  {"ψ_a = ?":<25}')
print('  ' + '-'*95)

for u, v, label in sorted(pi1_calc.Y.edges()):
    J_u = oid_to_J[u]
    J_v = oid_to_J[v]
    in_tree = frozenset([u, v]) in tree_upairs

    Ju_str = '{' + ','.join(f's{j}' for j in J_u) + '}' if J_u else '∅'
    Jv_str = '{' + ','.join(f's{j}' for j in J_v) + '}' if J_v else '∅'

    h = pi1_calc.h_a.get((u, v, label))
    if h is None:
        continue

    is_id    = (h == W.one())
    h_str    = 'e  (identidad)' if is_id else f'w (long. {h.length()})'
    tree_str = '✓ T'           if in_tree else '✗  (generador libre)'

    if is_id and in_tree:
        psi_str = 'inclusión  G_{J_u} ↪ G_{J_v}'
    elif is_id:
        psi_str = 'inclusión  (+ gen. libre)'
    else:
        psi_str = f'conjugación por h_a'

    print(f'  W_{Ju_str} → W_{Jv_str}   {tree_str:^10}   {h_str:^20}   {psi_str:<30}')

print()
print('Conclusión: todos los h_a = e en este ejemplo.')
print('Por tanto todos los ψ_a son simplemente inclusiones de grupos parabolics.')
print('Esto es consecuencia de que W actúa sobre K_A preservando los tipos J.')
print()
print('La elección del árbol T (verde en el diagrama) colapsa esas inclusiones')
print('en el grupo orbifold G_A, dejando sólo la estructura de G_A libre sobre T.')


---
## 3. Los Morfismos $\psi_a$ y el Elemento de Torsión

### La cara retenida $eW_{\{1,3\}}$ — donde nace la relación $(s_1 s_3)^2 = 1$

El grupo local de la cara retenida es $G_f = W_{\{1,3\}} = \langle s_1, s_3 \rangle \cong \mathbb{Z}/2 \times \mathbb{Z}/2$
(producto directo porque $m_{13} = 2$, i.e., $s_1 s_3 = s_3 s_1$).

Los morfismos de inclusión (para elementos de transporte $h_a = e$ en la mayoría de aristas):
$$\psi_{e_1 \to f}: \langle s_1 \rangle \hookrightarrow W_{\{1,3\}}, \quad s_1 \mapsto s_1$$
$$\psi_{e_3 \to f}: \langle s_3 \rangle \hookrightarrow W_{\{1,3\}}, \quad s_3 \mapsto s_3$$

El **colímite** de este diagrama en el grupo orbifold produce exactamente la relación
$(s_1 s_3)^2 = 1$ (pues en $W_{\{1,3\}}$ se tiene $(s_1 s_3)^2 = 1$).

Las caras **removidas** (hexagonos, tipo $\{1,2\}$ y $\{2,3\}$) no contribuyen relaciones,
dejando $m'_{12} = m'_{23} = \infty$.

In [ ]:
# Explicit psi morphisms for the retained face eW_{1,3}
J_face = (1, 3)   # the retained square
J_e1   = (1,)     # edge type {1}
J_e3   = (3,)     # edge type {3}

# Find canonical orbit ids
def find_orbit(pi1, w, J):
    canon = pi1.dga._canonize((w, tuple(sorted(J))))
    return pi1.orbits.get(canon)

oid_face = find_orbit(pi1_calc, e, J_face)
oid_e1   = find_orbit(pi1_calc, e, J_e1)
oid_e3   = find_orbit(pi1_calc, e, J_e3)

G_face = pi1_calc.stabilizers[oid_face]
G_e1   = pi1_calc.stabilizers[oid_e1]
G_e3   = pi1_calc.stabilizers[oid_e3]

print('--- Local Group at retained face eW_{1,3} ---')
print(f'G_f = Stab(eW_{{1,3}}) = {[str(g) for g in G_face]}')
print(f'Order: {len(G_face)}  (Z/2 × Z/2)')

print()
print('--- Local Groups at adjacent edges ---')
print(f'G_e1 = Stab(eW_{{1}}) = {[str(g) for g in G_e1]}')
print(f'G_e3 = Stab(eW_{{3}}) = {[str(g) for g in G_e3]}')

print()
print('--- psi morphisms (inclusions, transport h_a = e) ---')
print('psi_{e1 -> f}: s1 |--> s1  (includes <s1> into W_{1,3})')
print('psi_{e3 -> f}: s3 |--> s3  (includes <s3> into W_{1,3})')

print()
print('--- Torsion element from face: relation in G_A ---')
# The local group W_{1,3} imposes: (s1*s3)^2 = 1
s1 = sref[1]; s3 = sref[3]
print(f'In W_{{1,3}}: (s1*s3)^2 = {(s1*s3)**2 == e}  (identity?)')
print(f'Order of s1*s3 in W: {(s1*s3).order()}')
print(f'=> Relation added to G_A: (s1*s3)^2 = 1')

### Por contraste: las caras REMOVIDAS

Para el estrato $\{1,2\}$ (removido), el grupo local sería $W_{\{1,2\}} \cong S_3$
con $(s_1 s_2)^3 = 1$. Pero como la celda **no existe** en $K_{\mathscr{A}}$,
no hay morfismo de inclusión que fuerce esa relación: $m'_{12} = \infty$.

In [ ]:
# Verify: the rank-2 cells of type {1,2} are absent from dga.cells
J_hex = (1, 2)
cells_type_12 = [c for c in dga.cells if c[1] == J_hex]
cells_type_23 = [c for c in dga.cells if c[1] == (2, 3)]
cells_type_13 = [c for c in dga.cells if c[1] == (1, 3)]

print(f'Cells of type {{1,2}} in K_A: {len(cells_type_12)}  (removed - 0 expected)')
print(f'Cells of type {{2,3}} in K_A: {len(cells_type_23)}  (removed - 0 expected)')
print(f'Cells of type {{1,3}} in K_A: {len(cells_type_13)}  (retained - 6 expected)')

print()
s2 = sref[2]
print('Would-be relation (s1*s2)^3 = 1 from W_{1,2}:')
print(f'  (s1*s2)^3 = identity in W? {(s1*s2)**3 == e}')
print(f'  -> Since {{1,2}} is absent: no relation, m prime_12 = infinity')

---
## 4. El Grupo Orbifold $\mathcal{G}_{\mathscr{A}}$

Resumiendo la construcción B&H para este caso $W$-invariante:

$$\mathcal{G}_{\mathscr{A}} = \left\langle s_1, s_2, s_3 \;\middle|\;
\underbrace{s_i^2 = 1}_{\text{grupos locales en aristas}},\;
\underbrace{(s_1 s_3)^2 = 1}_{\text{cara retenida }\{1,3\}}
\right\rangle$$

La **matriz de Coxeter modificada** $m'$:

| Par | $m_{st}$ original | Estrato | $m'_{st}$ |
|-----|-------------------|---------|----------|
| $(1,2)$ | 3 | Removido | $\infty$ |
| $(2,3)$ | 3 | Removido | $\infty$ |
| $(1,3)$ | 2 | **Retenido** | **2** |

In [ ]:
print('Modified Coxeter data m prime_{st}:')
for (s, t), m in sorted(pi1_calc.modified_coxeter_data().items()):
    status = 'REMOVED' if m == Infinity else f'retained (m={m})'
    print(f'  m prime_{{{s},{t}}} = {m}   <-- {status}')

print()
G_A, _ = pi1_calc.orbifold_group_presentation(verbose=False)
print('G_A presentation:')
print('  Generators:', [str(g) for g in G_A.generators()])
print('  Relations:',  G_A.relations())
print()
print('Interpretation: G_A is an infinite Coxeter group.')
print('The only finite-order braid relation is (s1*s3)^2 = 1.')
print('s1*s2 and s2*s3 have INFINITE order in G_A.')

---
## 5. Cálculo de $\pi_1(K_{\mathscr{A}}) = \ker(\rho)$

El homomorfismo de monodromía $\rho: \mathcal{G}_{\mathscr{A}} \to W$ envía $s_i \mapsto s_i$.
Es sobreyectivo (los $s_i$ generan $W$) e $[G_A : \ker(\rho)] = |W| = 24$.

GAP computa $\ker(\rho)$ por enumeración de cosetes (Reidemeister–Schreier).
El rango libre debe coincidir con $\dim H^1 = 7$.

In [ ]:
# Use the simpler ParabolicArrangementPi1 (H=trivial, uses orbifold theorem)
pi1_simple = ParabolicArrangementPi1(dga)
K, phi = pi1_simple.compute_pi1(verbose=True)

In [ ]:
ab = list(K.AbelianInvariants())
free_rank = ab.count(0)
betti1 = len(dga.cohomology_basis(1, ring=GF(2)))

print(f'Abelian invariants of pi_1: {ab}')
print(f'Free rank of pi_1^ab:      {free_rank}')
print(f'dim H^1 (cohomology):      {betti1}')
print()
if free_rank == betti1 and not any(x > 0 for x in ab):
    print('VERIFIED: pi_1 is a FREE group of rank 7  (F_7)')
    print('This confirms BSW Theorem 6.1 for this arrangement.')

---
## 5.5. Verificación: Fórmula de Reidemeister–Schreier

La fórmula de Euler en la 1-esqueleto de $K_{\mathscr{A}}$ (el complejo del principio):

$$|\text{gen. libres de }\pi_1| \;=\; |E| - |V| + 1 \;=\; 36 - 24 + 1 \;=\; 13$$

Pero $K_{\mathscr{A}}$ también tiene 6 caras cuadradas, cada una impone una relación
$(s_1 s_3)^2 = 1$ en el espacio de lazos. Eso reduce el rango:

$$\pi_1(K_{\mathscr{A}}) \cong F_{13-6} = F_7$$

El scwol $Y$ y el árbol $T$ que determinan $\rho$ y $\pi_1$ se desarrollaron en la **Sección 2.5**.

---
## 6. Visualización 3D: Un Generador de $\pi_1$ en el Permutatoedro

El permutatoedro de $A_3$ es el **octaedro truncado** con vértices en las 24 permutaciones
de $(1,2,3,4) \in \mathbb{R}^4$, proyectado a $\mathbb{R}^3$ usando las raíces simples.

**Generador explícito de $\pi_1$:** el borde de la cara hexagonal removida $eW_{\{1,2\}}$:

$$\gamma = s_1 \cdot s_2 \cdot s_1 \cdot s_2 \cdot s_1 \cdot s_2 = (s_1 s_2)^3$$

En el permutatoedro **completo**, $(s_1 s_2)^3 = 1$ porque la celda hexagonal existe
y el lazo es contráctil. En $K_{\mathscr{A}}$ (sin la cara), este lazo es **no contráctil**:
es un generador de $\pi_1 \cong F_7$.

Nótese que $\rho((s_1 s_2)^3) = (s_1 s_2)^3|_W = \mathrm{id} \in W = S_4$
(pues $(s_1 s_2)^3 = 1$ en $W$), confirmando que es un elemento del kernel.

In [ ]:
# Verify the loop (s1*s2)^3 is in the kernel of rho: G_A -> W
print('Loop gamma = (s1*s2)^3:')
print(f'  In W = S_4: (s1*s2)^3 = {(s1*s2)**3} = identity?', (s1*s2)**3 == e)
print(f'  In G_A (no relation): (s1*s2)^3 has INFINITE order')
print(f'  => gamma lies in ker(rho) = pi_1(K_A)')
print()

# The 6 vertices of the hexagonal face eW_{1,2} as permutations of (1,2,3,4)
# s_i = transposition swapping entries at positions i and i+1
def apply_s(perm, i):
    p = list(perm)
    p[i-1], p[i] = p[i], p[i-1]
    return tuple(p)

# Build the hexagonal loop: e, s1, s1*s2, s1*s2*s1, s2*s1, s2, back to e
start = (1,2,3,4)
loop = [start]
steps = [1,2,1,2,1,2]  # alternating s1, s2
cur = start
for si in steps:
    cur = apply_s(cur, si)
    loop.append(cur)

print('Hexagonal loop (s1*s2)^3 in one-line notation:')
for i, (v, si) in enumerate(zip(loop[:-1], steps + ['done'])):
    arrow = f' --s{si}--> ' if si != 'done' else ''
    print(f'  {list(v)}{arrow}', end='')
    if (i+1) % 3 == 0: print()
print()
print(f'Loop closes: {loop[0] == loop[-1]}')

In [ ]:
import numpy as np
from itertools import permutations as iperms

# Redefine geometry with the cube-axes ONB (same as cell 92034e2d above).
# All three axes give range [−2, 2], so aspectmode='cube' shows an isotropic shape.
c1 = np.array([ 1,-1,-1, 1], dtype=float) / 2
c2 = np.array([ 1, 1,-1,-1], dtype=float) / 2
c3 = np.array([ 1,-1, 1,-1], dtype=float) / 2

all_verts = sorted(set(iperms([1,2,3,4])))
center_4d = np.mean(np.array(all_verts, dtype=float), axis=0)

def proj(v):
    cv = np.array(v, dtype=float) - center_4d
    return np.array([cv @ c1, cv @ c2, cv @ c3])

coords = {v: proj(v) for v in all_verts}

def edge_type(u, v):
    diff = [i for i in range(4) if u[i] != v[i]]
    if len(diff) == 2 and abs(diff[0]-diff[1]) == 1:
        if u[diff[0]] == v[diff[1]] and u[diff[1]] == v[diff[0]]:
            return diff[0] + 1
    return None

edges = []
for i, u in enumerate(all_verts):
    for v in all_verts[i+1:]:
        et = edge_type(u, v)
        if et is not None:
            edges.append((u, v, et))

print(f'Vertices: {len(all_verts)}')
print(f'Edges:    {len(edges)}')
print(f'Edges by type: s1={sum(1 for e in edges if e[2]==1)},',
      f's2={sum(1 for e in edges if e[2]==2)},',
      f's3={sum(1 for e in edges if e[2]==3)}')


In [ ]:
# ── Verification: edges from edge_type  ==  Sage convex hull 1-skeleton ───────
P_check = Polyhedron(vertices=all_verts)

# Collect edge pairs from Sage's 1-faces
sage_edge_pairs = set()
for f in P_check.faces(1):
    vts = [tuple(int(x) for x in v.vector()) for v in f.vertices()]
    assert len(vts) == 2
    sage_edge_pairs.add(frozenset(vts))

# Collect edge pairs from our edge_type function
our_edge_pairs = {frozenset([u, v]) for u, v, _ in edges}

ok = (our_edge_pairs == sage_edge_pairs)
print(f'Sage convex hull 1-faces:  {len(sage_edge_pairs)} edges')
print(f'Our edge_type edges:       {len(our_edge_pairs)} edges')
print(f'Sets equal:  {ok}  ← {"✓ CORRECTO" if ok else "✗ MISMATCH"}')
print()
print('Nota: Las aristas que parecen "atravesar" el sólido en plotly')
print('son aristas de la cara trasera visibles a través de las caras')
print('semi-transparentes (opacity < 1). Es un artefacto de renderizado:')
print('todas las aristas están sobre la superficie del octaedro truncado.')


In [ ]:
import plotly.graph_objects as go
import numpy as np
from itertools import permutations as iperms
from sage.geometry.polyhedron.constructor import Polyhedron

# ── Geometry (self-contained) ────────────────────────────────────────────────
all_verts = sorted(set(iperms([1,2,3,4])))
center_4d = np.mean(np.array(all_verts, dtype=float), axis=0)

c1 = np.array([ 1,-1,-1, 1], dtype=float) / 2
c2 = np.array([ 1, 1,-1,-1], dtype=float) / 2
c3 = np.array([ 1,-1, 1,-1], dtype=float) / 2

def proj(v):
    cv = np.array(v, dtype=float) - center_4d
    return np.array([cv @ c1, cv @ c2, cv @ c3])

coords = {v: proj(v) for v in all_verts}

# ── Edge typing: pos-swap vs val-swap (choose the one that matches the 1-skeleton)

def edge_type_pos(u, v):
    diff = [i for i in range(4) if u[i] != v[i]]
    if len(diff) == 2 and abs(diff[0]-diff[1]) == 1:
        if u[diff[0]] == v[diff[1]] and u[diff[1]] == v[diff[0]]:
            return diff[0] + 1
    return None


def edge_type_val(u, v):
    for i in (1, 2, 3):
        ulist = list(u)
        pi = ulist.index(i)
        pj = ulist.index(i + 1)
        ulist[pi], ulist[pj] = ulist[pj], ulist[pi]
        if tuple(ulist) == v:
            return i
    return None

P_loc      = Polyhedron(vertices=all_verts)
sage_hex_l = [f for f in P_loc.faces(2) if f.n_vertices() == 6]
sage_sq_l  = [f for f in P_loc.faces(2) if f.n_vertices() == 4]

edge_pairs = []
for f in P_loc.faces(1):
    vts = [tuple(int(x) for x in v.vector()) for v in f.vertices()]
    edge_pairs.append((vts[0], vts[1]))

def classify_edges(edge_pairs, edge_type_fn):
    counts = {1: 0, 2: 0, 3: 0, None: 0}
    typed = []
    for u, v in edge_pairs:
        t = edge_type_fn(u, v)
        counts[t] = counts.get(t, 0) + 1
        typed.append((u, v, t))
    return typed, counts

edge_pos, counts_pos = classify_edges(edge_pairs, edge_type_pos)
edge_val, counts_val = classify_edges(edge_pairs, edge_type_val)

if counts_pos.get(None, 0) == 0:
    edges = edge_pos
    edge_mode = 'posiciones'
elif counts_val.get(None, 0) == 0:
    edges = edge_val
    edge_mode = 'valores'
else:
    edges = edge_pos
    edge_mode = 'posiciones (con None)'

print('Edges total:', len(edge_pairs))
print('Counts pos-swap:', counts_pos)
print('Counts val-swap:', counts_val)
print('Usando modo:', edge_mode)

# ── Loop γ = (s1 s2)^3 consistent with edge_mode ─────────────────────────────

def apply_s_pos(perm, i):
    p = list(perm)
    p[i-1], p[i] = p[i], p[i-1]
    return tuple(p)


def apply_s_val(perm, i):
    p = list(perm)
    pi = p.index(i)
    pj = p.index(i + 1)
    p[pi], p[pj] = p[pj], p[pi]
    return tuple(p)

apply_s = apply_s_pos if edge_mode.startswith('posiciones') else apply_s_val

start = (1, 2, 3, 4)
loop = [start]
steps = [1, 2, 1, 2, 1, 2]
cur = start
for si in steps:
    cur = apply_s(cur, si)
    loop.append(cur)

# ── helpers ──────────────────────────────────────────────────────────────────

def _face_pts(f):
    vt  = [tuple(int(x) for x in v.vector()) for v in f.vertices()]
    pts = np.array([coords[v] for v in vt])
    cen = pts.mean(axis=0)
    e1  = pts[1] - pts[0]; e1 /= np.linalg.norm(e1)
    nrm = np.cross(pts[1]-pts[0], pts[2]-pts[0]); nrm /= np.linalg.norm(nrm)
    e2  = np.cross(nrm, e1)
    return pts[np.argsort(np.arctan2([(p-cen)@e2 for p in pts],
                                     [(p-cen)@e1 for p in pts]))]

def _tri(face_list):
    vx, vy, vz, fi, fj, fk = [], [], [], [], [], []
    for f in face_list:
        pts = _face_pts(f); n = len(pts); base = len(vx)
        for p in pts:
            vx.append(float(p[0])); vy.append(float(p[1])); vz.append(float(p[2]))
        for t in range(1, n-1):
            fi.append(base); fj.append(base+t); fk.append(base+t+1)
    return vx, vy, vz, fi, fj, fk

_fl = dict(ambient=1.0, diffuse=0.0, specular=0.0, roughness=1.0, fresnel=0.0)
FACE_OPACITY = 0.65
EDGE_OFFSET = 0.02

# Faces first so edges render on top
traces = []

# ALL 14 faces shown — closed surface so no back-edge artifacts
sq_vx, sq_vy, sq_vz, sq_fi, sq_fj, sq_fk = _tri(sage_sq_l)
traces.append(go.Mesh3d(x=sq_vx, y=sq_vy, z=sq_vz, i=sq_fi, j=sq_fj, k=sq_fk,
    color='#3182bd', opacity=FACE_OPACITY, lighting=_fl,
    name='Cara cuadrada (retenida, m₁₃=2)', showlegend=True))

hx_vx, hx_vy, hx_vz, hx_fi, hx_fj, hx_fk = _tri(sage_hex_l)
traces.append(go.Mesh3d(x=hx_vx, y=hx_vy, z=hx_vz, i=hx_fi, j=hx_fj, k=hx_fk,
    color='#d9d9d9', opacity=FACE_OPACITY, lighting=_fl,
    name='Cara hexagonal (removida, gris)', showlegend=True))

# ── edges by generator ────────────────────────────────────────────────────────
ec = {1: '#e05252', 2: '#52a852', 3: '#5252e0'}
en = {1: 's₁-aristas', 2: 's₂-aristas', 3: 's₃-aristas'}
extra_x, extra_y, extra_z = [], [], []

for et in [1, 2, 3]:
    xs, ys, zs = [], [], []
    for u, v, t in edges:
        if t != et:
            continue
        off = np.array([0.0, 0.0, 0.0])
        pu, pv = coords[u] + off, coords[v] + off
        xs += [float(pu[0]), float(pv[0]), None]
        ys += [float(pu[1]), float(pv[1]), None]
        zs += [float(pu[2]), float(pv[2]), None]
    traces.append(go.Scatter3d(x=xs, y=ys, z=zs, mode='lines',
        line=dict(color=ec[et], width=4), name=en[et]))

for u, v, t in edges:
    if t is not None:
        continue
    pu, pv = coords[u], coords[v]
    extra_x += [float(pu[0]), float(pv[0]), None]
    extra_y += [float(pu[1]), float(pv[1]), None]
    extra_z += [float(pu[2]), float(pv[2]), None]

if extra_x:
    traces.append(go.Scatter3d(
        x=extra_x, y=extra_y, z=extra_z, mode='lines',
        line=dict(color='#111111', width=4), name='aristas sin tipo'))

# ── loop γ = (s₁s₂)³ ─────────────────────────────────────────────────────────
lp = np.array([coords[v] for v in loop])
traces.append(go.Scatter3d(
    x=lp[:,0].tolist(), y=lp[:,1].tolist(), z=lp[:,2].tolist(),
    mode='lines+markers',
    line=dict(color='#ff8c00', width=9),
    marker=dict(size=9, color='#ff8c00', line=dict(color='black', width=1)),
    name='γ = (s₁s₂)³  generador de π₁'))

# ── vertices ──────────────────────────────────────────────────────────────────
loop_set = set(loop[:-1])
vc   = ['#ff8c00' if v in loop_set else '#555555' for v in all_verts]
vs_  = [10        if v in loop_set else 4          for v in all_verts]
allp = np.array([coords[v] for v in all_verts])
traces.append(go.Scatter3d(
    x=allp[:,0].tolist(), y=allp[:,1].tolist(), z=allp[:,2].tolist(),
    mode='markers', marker=dict(size=vs_, color=vc, opacity=0.9),
    text=[str(list(v)) for v in all_verts], hoverinfo='text', name='Vértices w ∈ W'))

# ── layout ────────────────────────────────────────────────────────────────────
_ax = dict(visible=False)
fig = go.Figure(data=traces)
fig.update_layout(
    title=dict(
        text=('Octaedro Truncado A₃ — K<sub>𝒜</sub><br>'
              f'<sup>Lazo naranja (s₁s₂)³ en modo {edge_mode} | '
              'azul = cuadrado retenido | gris = hexágono removido</sup>'),
        font=dict(size=13)),
    scene=dict(xaxis=_ax, yaxis=_ax, zaxis=_ax,
               bgcolor='white', aspectmode='cube',
               camera=dict(eye=dict(x=2.5, y=0.3, z=0.3))),
    legend=dict(x=0.01, y=0.99, bgcolor='rgba(255,255,255,0.8)', font=dict(size=10)),
    margin=dict(l=0, r=0, b=0, t=70), width=820, height=660)

fig.write_html('a3_pi1_generator.html')
print('Interactive HTML saved to  a3_pi1_generator.html')
fig.show()



---
## 6.5. Árbol Maximal $T$ en $K_{\mathscr{A}}$ — De Dónde Viene $F_7$

Para ver **por qué** $\pi_1 \cong F_7$, miramos la 1-esqueleto de $K_{\mathscr{A}}$
como grafo: 24 vértices y 36 aristas.

**Construcción de $\pi_1$ vía van Kampen / Reidemeister–Schreier:**

1. **Árbol maximal $T$** (verde): 23 aristas conectan los 24 vértices sin ciclos.
2. **Aristas no-árbol** (rojo): $36 - 23 = 13$ aristas, cada una da un generador
   de $\pi_1(\text{1-esqueleto}) \cong F_{13}$.
3. **Caras cuadradas** (azul): cada cuadrado retenido impone una relación en $\pi_1$.
   Las 6 caras cuadradas matan 6 generadores.
4. **Resultado**: $\pi_1(K_{\mathscr{A}}) \cong F_{13-6} = F_7$.

Las caras hexagonales **removidas** son precisamente las que hacen que los lazos
$(s_i s_j)^3$ sean no contráctiles — 8 hexágonos potenciales, pero sólo 7 son
independientes (el producto de todos es la identidad en $\pi_1$).

In [ ]:
import plotly.graph_objects as go
import numpy as np
from collections import deque
from sage.geometry.polyhedron.constructor import Polyhedron

# ── BFS spanning tree ─────────────────────────────────────────────────────────
adj = {v: [] for v in all_verts}
for u, v, et in edges:
    adj[u].append((v, et))
    adj[v].append((u, et))

root = (1, 2, 3, 4)
visited, tree_vpairs = set(), set()
bfs_queue = deque([root])
visited.add(root)
while bfs_queue:
    node = bfs_queue.popleft()
    for nbr, et in adj[node]:
        if nbr not in visited:
            visited.add(nbr)
            tree_vpairs.add(frozenset([node, nbr]))
            bfs_queue.append(nbr)

tree_edges_list = [(u, v, et) for u, v, et in edges if frozenset([u, v]) in  tree_vpairs]
non_tree        = [(u, v, et) for u, v, et in edges if frozenset([u, v]) not in tree_vpairs]

print(f'1-skeleton:    {len(all_verts)} vértices,  {len(edges)} aristas')
print(f'Árbol T:       {len(tree_edges_list)} aristas  (|V|-1 = {len(all_verts)-1})')
print(f'No-árbol:      {len(non_tree)} aristas  →  F_{len(non_tree)}')

# ── All faces of truncated octahedron ─────────────────────────────────────────
P_t       = Polyhedron(vertices=all_verts)
sage_sq_t = [f for f in P_t.faces(2) if f.n_vertices() == 4]
sage_hx_t = [f for f in P_t.faces(2) if f.n_vertices() == 6]

def _face_pts_t(f):
    vt  = [tuple(int(x) for x in v.vector()) for v in f.vertices()]
    pts = np.array([coords[v] for v in vt])
    cen = pts.mean(axis=0)
    e1  = pts[1] - pts[0]; e1 /= np.linalg.norm(e1)
    nrm = np.cross(pts[1]-pts[0], pts[2]-pts[0]); nrm /= np.linalg.norm(nrm)
    e2  = np.cross(nrm, e1)
    return pts[np.argsort(np.arctan2([(p-cen)@e2 for p in pts],
                                     [(p-cen)@e1 for p in pts]))]

def _tri_t(face_list):
    vx, vy, vz, fi, fj, fk = [], [], [], [], [], []
    for f in face_list:
        pts = _face_pts_t(f); n = len(pts); base = len(vx)
        for p in pts:
            vx.append(float(p[0])); vy.append(float(p[1])); vz.append(float(p[2]))
        for t in range(1, n-1):
            fi.append(base); fj.append(base+t); fk.append(base+t+1)
    return vx, vy, vz, fi, fj, fk

# ── Traces: ALL faces first (closed solid), then edges on top ─────────────────
traces_t = []
_fl_t = dict(ambient=1.0, diffuse=0.0, specular=0.0, roughness=1.0, fresnel=0.0)

sq_tvx, sq_tvy, sq_tvz, sq_tfi, sq_tfj, sq_tfk = _tri_t(sage_sq_t)
traces_t.append(go.Mesh3d(x=sq_tvx, y=sq_tvy, z=sq_tvz,
    i=sq_tfi, j=sq_tfj, k=sq_tfk,
    color='#3182bd', opacity=1.0, lighting=_fl_t,
    name=f'{len(sage_sq_t)} cuadrados  (→ −{len(sage_sq_t)} relaciones)'))

hx_tvx, hx_tvy, hx_tvz, hx_tfi, hx_tfj, hx_tfk = _tri_t(sage_hx_t)
traces_t.append(go.Mesh3d(x=hx_tvx, y=hx_tvy, z=hx_tvz,
    i=hx_tfi, j=hx_tfj, k=hx_tfk,
    color='#d9d9d9', opacity=1.0, lighting=_fl_t,
    name=f'{len(sage_hx_t)} hexágonos (gris)'))

tc = {1: '#1a9850', 2: '#74c476', 3: '#006d2c'}
for et in [1, 2, 3]:
    xs, ys, zs = [], [], []
    for u, v, t in tree_edges_list:
        if t == et:
            pu, pv = coords[u], coords[v]
            xs += [float(pu[0]), float(pv[0]), None]
            ys += [float(pu[1]), float(pv[1]), None]
            zs += [float(pu[2]), float(pv[2]), None]
    if xs:
        traces_t.append(go.Scatter3d(x=xs, y=ys, z=zs, mode='lines',
            line=dict(color=tc[et], width=5), name=f'Árbol s{et}'))

ntx, nty, ntz = [], [], []
for u, v, _ in non_tree:
    pu, pv = coords[u], coords[v]
    ntx += [float(pu[0]), float(pv[0]), None]
    nty += [float(pu[1]), float(pv[1]), None]
    ntz += [float(pu[2]), float(pv[2]), None]
traces_t.append(go.Scatter3d(x=ntx, y=nty, z=ntz, mode='lines', opacity=0.85,
    line=dict(color='#d62728', width=3),
    name=f'{len(non_tree)} aristas no-árbol  (→ F_{len(non_tree)})'))

allp     = np.array([coords[v] for v in all_verts])
root_clr = ['#ff7f0e' if v == root else '#333333' for v in all_verts]
root_sz  = [10        if v == root else 4          for v in all_verts]
traces_t.append(go.Scatter3d(
    x=allp[:,0].tolist(), y=allp[:,1].tolist(), z=allp[:,2].tolist(),
    mode='markers',
    marker=dict(size=root_sz, color=root_clr, opacity=0.9,
                line=dict(color='black', width=0.5)),
    text=[str(list(v)) for v in all_verts], hoverinfo='text',
    name='Vértices  (naranja = raíz BFS)'))

# ── Layout ────────────────────────────────────────────────────────────────────
_ax = dict(visible=False)
n_nt, n_sq = len(non_tree), len(sage_sq_t)
fig_tree = go.Figure(data=traces_t)
fig_tree.update_layout(
    title=dict(
        text=(f'Árbol maximal T en K<sub>𝒜</sub>  (BFS desde [1,2,3,4])<br>'
              f'<sup>Verde = árbol ({len(tree_edges_list)} aristas) | '
              f'Rojo = {n_nt} no-árbol → F<sub>{n_nt}</sub> | '
              f'Azul = {n_sq} caras → π₁ = F<sub>{n_nt-n_sq}</sub></sup>'),
        font=dict(size=12)),
    scene=dict(xaxis=_ax, yaxis=_ax, zaxis=_ax,
               bgcolor='white', aspectmode='cube',
               camera=dict(eye=dict(x=2.5, y=0.3, z=0.3))),
    legend=dict(x=0.01, y=0.99, bgcolor='rgba(255,255,255,0.85)', font=dict(size=10)),
    margin=dict(l=0, r=0, b=0, t=90), width=820, height=660)

fig_tree.write_html('a3_spanning_tree.html')
print(f'Caras retain.: {n_sq} cuadrados  →  F_{n_nt-n_sq}  ✓')
print('Saved  a3_spanning_tree.html')
fig_tree.show()


---
---

## Part B: The $H$-Invariant Case and Cross-Validation

## 8. The Subgroup $H = \langle s_1, s_2 \rangle$ and $H$-Invariance

We fix $H = \langle s_1, s_2 \rangle \le W$. This is a dihedral subgroup of order 6
(isomorphic to $S_3$, generated by a simple reflection and an adjacent one).

Because the arrangement is $W$-invariant, it is automatically $H$-invariant for every $H \le W$.
This lets us test the full Bridson-Haefliger machinery with the smaller group $H$ and compare.

In [ ]:
# Helper for word notation
def word_str(g):
    if g == W.one():
        return 'e'
    try:
        return '*'.join(f's{i}' for i in g.reduced_word())
    except:
        return str(g)

H_gens = [sref[1], sref[2]]
H = W.subgroup(H_gens)

print('H generators:', [word_str(g) for g in H_gens])
print('|H| =', len(H))

# Verify H-invariance of Delta
delta_can = {dga._canonize(c) for c in Delta if dga._is_valid_cell(c)}
h_inv = True
for h in H:
    hw = W(h)
    for c in delta_can:
        w, J = c
        if dga._canonize((hw * w, J)) not in delta_can:
            h_inv = False
            break
    if not h_inv:
        break
print(f'Delta is H-invariant: {h_inv}')

In [ ]:
# Build complex of groups with H = <s1, s2>
pi1_demo = ParabolicArrangementPi1(dga, H_gens=H_gens)
print(f'Quotient scwol Y: {pi1_demo.Y.num_verts()} objects, {pi1_demo.Y.num_edges()} morphisms')

### Orbits, Stabilizers, and the Quotient Scwol $\mathcal{Y}$

Each object of $\mathcal{Y}$ is an $H$-orbit of cells in $K_{\mathscr{A}}$. The table below
shows, for each orbit, its dimension (= $|J|$), representative cell, and local stabilizer.

In [ ]:
from collections import defaultdict

# --- Table: one row per orbit ---
print(f'{"oid":>8}  {"dim":>3}  {"J":<9}  {"orbit":>5}  {"|Stab|":>6}  representative')
print('-' * 65)
orbit_info = []
for oid, rep in pi1_demo.reps.items():
    w, J = rep
    orb_size = sum(1 for c in pi1_demo.orbits if pi1_demo.orbits[c] == oid)
    stab_size = len(pi1_demo.stabilizers[oid])
    orbit_info.append((len(J), J, oid, orb_size, stab_size, w))

orbit_info.sort(key=lambda x: (x[0], x[1], x[2]))
for dim, J, oid, orb_size, stab_size, w in orbit_info:
    print(f'{oid:>8}  {dim:>3}  {str(J):<9}  {orb_size:>5}  {stab_size:>6}  ({word_str(w)}, {J})')

# --- Summary by parabolic type ---
print()
print('Summary by parabolic type J:')
print(f'{"J":<10}  {"#orbits":>7}  {"orbit sizes":<22}  {"|Stab| per orbit":<18}  total')
print('-' * 75)
orbit_info_by_J = defaultdict(list)
for oid, rep in pi1_demo.reps.items():
    _, J = rep
    orb_size = sum(1 for c in pi1_demo.orbits if pi1_demo.orbits[c] == oid)
    stab_size = len(pi1_demo.stabilizers[oid])
    orbit_info_by_J[J].append((oid, orb_size, stab_size))

for J in sorted(orbit_info_by_J.keys(), key=lambda j: (len(j), j)):
    infos = orbit_info_by_J[J]
    sizes = [o[1] for o in infos]
    stabs = [o[2] for o in infos]
    total_J = sum(1 for c in dga.cells if c[1] == J)
    print(f'{str(J):<10}  {len(infos):>7}  {str(sizes):<22}  {str(stabs):<18}  {total_J}')

print()
print('Note: orbit size × |H| / |Stab| = |H| always holds (orbit-stabilizer theorem).')


**Hasse diagram of the quotient scwol $\mathcal{Y} = H \backslash \mathcal{X}_{\mathscr{A}}$.**
Each node represents one $H$-orbit; edges are the morphisms of $\mathcal{Y}$.
Tree edges (part of the spanning tree $T$) are solid; non-tree edges are dashed.

In [ ]:
# Hasse diagram of the quotient scwol Y = H\X_A
import plotly.graph_objects as go

J_to_pos = {
    ():     (2.0, 0.0),
    (1,):   (0.5, 1.0),
    (2,):   (2.0, 1.0),
    (3,):   (3.5, 1.0),
    (1, 3): (2.0, 2.0),
}

oid_to_J   = {oid: J for oid, (_, J) in pi1_demo.reps.items()}

# Spread multiple orbits of the same J to avoid overlap
orbits_by_J = {}
for oid, J in oid_to_J.items():
    orbits_by_J.setdefault(J, []).append(oid)
for J in orbits_by_J:
    orbits_by_J[J].sort()

# Place each level (by |J|) on its own y, and spread widely in x
level_to_oids = {}
for J, oids in orbits_by_J.items():
    level_to_oids.setdefault(len(J), []).extend(oids)
for level in level_to_oids:
    level_to_oids[level].sort()

x_gap = 1.6  # larger spacing between orbits
x_offset = 0.0

oid_to_pos = {}
for level, oids in sorted(level_to_oids.items()):
    k = len(oids)
    if k == 1:
        oid_to_pos[oids[0]] = (x_offset, float(level))
        continue
    start = -x_gap * (k - 1) / 2.0
    for i, oid in enumerate(oids):
        oid_to_pos[oid] = (x_offset + start + i * x_gap, float(level))

undirected_Y   = pi1_demo.Y.to_undirected()
tree_edge_list = undirected_Y.min_spanning_tree(algorithm='Kruskal')
tree_upairs    = {frozenset([u, v]) for u, v, _ in tree_edge_list}

all_Y_edges = list(pi1_demo.Y.edges())
n_nontree   = sum(1 for u, v, _ in all_Y_edges if frozenset([u, v]) not in tree_upairs)

PAD = float(0.18)
tr_x, tr_y = [], []
nt_x, nt_y = [], []
annotations = []
for u, v, label in all_Y_edges:
    x0, y0 = oid_to_pos[u]; x1, y1 = oid_to_pos[v]
    in_tree = frozenset([u, v]) in tree_upairs
    color   = '#2ca02c' if in_tree else '#d62728'
    width   = float(3.0) if in_tree else float(2.0)
    dx, dy  = x1-x0, y1-y0
    nrm     = (dx**2+dy**2)**0.5 or 1.0
    pad = min(PAD, 0.25 * nrm)
    x0p, y0p = x0+pad*dx/nrm, y0+pad*dy/nrm
    x1p, y1p = x1-pad*dx/nrm, y1-pad*dy/nrm

    if in_tree:
        tr_x += [float(x0p), float(x1p), None]
        tr_y += [float(y0p), float(y1p), None]
    else:
        nt_x += [float(x0p), float(x1p), None]
        nt_y += [float(y0p), float(y1p), None]

    annotations.append(dict(
        ax=float(x0p), ay=float(y0p),
        x =float(x1p), y =float(y1p),
        xref='x', yref='y', axref='x', ayref='y',
        showarrow=True, arrowhead=int(2), arrowsize=float(1.3),
        arrowwidth=float(width), arrowcolor=color, opacity=0.9))

nx_, ny_, txt_ = [], [], []
for oid in sorted(pi1_demo.reps):
    J = oid_to_J[oid]
    x, y = oid_to_pos[oid]
    nx_.append(float(x)); ny_.append(float(y))
    Jlbl = '{' + ','.join(f's{j}' for j in J) + '}' if J else '∅'
    stab_order = len(pi1_demo.stabilizers[oid])
    txt_.append(f'eW<sub>{Jlbl}</sub><br>G={stab_order}<br>{oid}')

edge_tree = go.Scatter(x=tr_x, y=tr_y, mode='lines',
    line=dict(color='#2ca02c', width=float(3)), name='Árbol T')
edge_nt = go.Scatter(x=nt_x, y=nt_y, mode='lines',
    line=dict(color='#d62728', width=float(2)), name=f'{n_nontree} no-árbol')
node_tr  = go.Scatter(x=nx_, y=ny_, mode='markers+text',
    marker=dict(size=int(34), color='#aec7e8', line=dict(color='#1f77b4', width=int(2))),
    text=txt_, textposition='middle center', textfont=dict(size=int(8)),
    hoverinfo='skip', showlegend=False)

fig_Y = go.Figure(data=[edge_tree, edge_nt, node_tr])
fig_Y.update_layout(
    title=dict(text='Quotient scwol Y = H∖X<sub>𝒜</sub>,  H = ⟨s₁,s₂⟩', font=dict(size=12)),
    annotations=annotations,
    xaxis=dict(range=[float(-5.0), float(5.0)], showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(range=[float(-0.6), float(2.8)], showgrid=False, zeroline=False, showticklabels=False),
    plot_bgcolor='#f8f9fa', width=int(920), height=int(480),
    margin=dict(l=int(20), r=int(20), t=int(70), b=int(20)),
    legend=dict(x=float(0.01), y=float(0.01), bgcolor='rgba(255,255,255,0.85)'))

fig_Y.write_html('a3_scwol_Y_H_s1s2.html')
print(f'Y: {pi1_demo.Y.num_verts()} objetos | {pi1_demo.Y.num_edges()} morfismos | {len(tree_edge_list)} en árbol T | {n_nontree} fuera de T')
fig_Y.show()




**The restricted complex $K_{\mathscr{A}}$ with cells coloured by $H$-orbit.**
Vertices, edges, and squares of the same colour belong to the same orbit.
This colouring is exactly what the quotient $\mathcal{Y}$ captures combinatorially.

In [ ]:
# K_A with cells coloured by H-orbit (right action = swap positions preserves geometry)
import numpy as np
import plotly.graph_objects as go
from itertools import permutations as iperms
from sage.geometry.polyhedron.constructor import Polyhedron

# Geometry
all_verts = sorted(set(iperms([1,2,3,4])))
center_4d = np.mean(np.array(all_verts, dtype=float), axis=0)
c1 = np.array([ 1,-1,-1, 1], dtype=float) / 2
c2 = np.array([ 1, 1,-1,-1], dtype=float) / 2
c3 = np.array([ 1,-1, 1,-1], dtype=float) / 2
def proj(v):
    cv = np.array(v, dtype=float) - center_4d
    return np.array([cv @ c1, cv @ c2, cv @ c3])
coords = {v: proj(v) for v in all_verts}

# 1-skeleton from Sage
P = Polyhedron(vertices=all_verts)
edge_pairs = []
for f in P.faces(1):
    vts = [tuple(int(x) for x in v.vector()) for v in f.vertices()]
    edge_pairs.append(tuple(sorted(vts)))

# Square faces (4 vertices)
square_faces = []
for f in P.faces(2):
    if f.n_vertices() != 4:
        continue
    vts = [tuple(int(x) for x in v.vector()) for v in f.vertices()]
    square_faces.append(tuple(sorted(vts)))

# H-orbits via right action
H = W.subgroup(H_gens)
H_words = [list(W(h).reduced_word()) for h in H]

def apply_s_pos(perm, i):
    p = list(perm)
    p[i-1], p[i] = p[i], p[i-1]
    return tuple(p)

def apply_word(perm, word):
    cur = perm
    for i in word:
        cur = apply_s_pos(cur, i)
    return cur

def act_on_face(face, word):
    return tuple(sorted(apply_word(v, word) for v in face))

# Vertex orbits
vertex_orbit_id = {}
vertex_orbits = []
for v in all_verts:
    if v in vertex_orbit_id:
        continue
    orb = set()
    for word in H_words:
        orb.add(apply_word(v, word))
    oid = len(vertex_orbits)
    for u in orb:
        vertex_orbit_id[u] = oid
    vertex_orbits.append(sorted(orb))

# Edge orbits
edge_orbit_id = {}
edge_orbits = []
for e in edge_pairs:
    if e in edge_orbit_id:
        continue
    orb = set()
    for word in H_words:
        u = apply_word(e[0], word)
        v = apply_word(e[1], word)
        orb.add(tuple(sorted((u, v))))
    oid = len(edge_orbits)
    for ee in orb:
        edge_orbit_id[ee] = oid
    edge_orbits.append(sorted(orb))

# Square orbits
face_orbit_id = {}
face_orbits = []
for face in square_faces:
    if face in face_orbit_id:
        continue
    orb = set()
    for word in H_words:
        orb.add(act_on_face(face, word))
    oid = len(face_orbits)
    for f2 in orb:
        face_orbit_id[f2] = oid
    face_orbits.append(sorted(orb))

print(f'Vertex orbits: {len(vertex_orbits)},  Edge orbits: {len(edge_orbits)},  Square orbits: {len(face_orbits)}')



# Colour palette
palette = [
    '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
    '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
    '#bcbd22', '#17becf', '#8dd3c7', '#fb8072'
]
def col(oid):
    return palette[oid % len(palette)]

def ordered_cycle(vt_list):
    pts = np.array([coords[v] for v in vt_list])
    cen = pts.mean(axis=0)
    e1  = pts[1] - pts[0]; e1 /= np.linalg.norm(e1)
    nrm = np.cross(pts[1]-pts[0], pts[2]-pts[0]); nrm /= np.linalg.norm(nrm)
    e2  = np.cross(nrm, e1)
    order = np.argsort(np.arctan2([(p-cen)@e2 for p in pts],
                                  [(p-cen)@e1 for p in pts]))
    return [vt_list[i] for i in order]

# --- Figure: K_A with coloured orbits ---
traces = []

# Square faces (semi-transparent, coloured by orbit)
_flat = dict(ambient=1.0, diffuse=0.0, specular=0.0, roughness=1.0, fresnel=0.0)
FACE_OPACITY = 0.35

for oid, orb in enumerate(face_orbits):
    for face in orb:
        cycle = ordered_cycle(list(face))
        pts = np.array([coords[v] for v in cycle])
        vx, vy, vz = pts[:,0].tolist(), pts[:,1].tolist(), pts[:,2].tolist()
        fi, fj, fk = [0,0], [1,2], [2,3]
        traces.append(go.Mesh3d(
            x=vx, y=vy, z=vz, i=fi, j=fj, k=fk,
            color=col(oid), opacity=FACE_OPACITY, lighting=_flat,
            name=f'square orbit #{oid}', showlegend=False))

# Edges coloured by orbit
for oid, orb in enumerate(edge_orbits):
    xs, ys, zs = [], [], []
    for u, v in orb:
        pu, pv = coords[u], coords[v]
        xs += [float(pu[0]), float(pv[0]), None]
        ys += [float(pu[1]), float(pv[1]), None]
        zs += [float(pu[2]), float(pv[2]), None]
    traces.append(go.Scatter3d(
        x=xs, y=ys, z=zs, mode='lines',
        line=dict(color=col(oid), width=4),
        name=f'edge orbit #{oid}', showlegend=False))

# Vertices colored by orbit
vx, vy, vz, vc = [], [], [], []
for v in all_verts:
    p = coords[v]
    vx.append(float(p[0])); vy.append(float(p[1])); vz.append(float(p[2]))
    vc.append(col(vertex_orbit_id[v]))
traces.append(go.Scatter3d(
    x=vx, y=vy, z=vz, mode='markers',
    marker=dict(size=5, color=vc, line=dict(color='black', width=0.3)),
    text=[str(list(v)) for v in all_verts], hoverinfo='text',
    name='vertices (orbits)', showlegend=False))

fig = go.Figure(data=traces)
fig.update_layout(
    title=dict(text='K_A: H-orbits on vertices, edges, and squares', font=dict(size=12)),
    scene=dict(xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False),
               bgcolor='white', aspectmode='cube',
               camera=dict(eye=dict(x=2.5, y=0.3, z=0.3))),
    margin=dict(l=0, r=0, b=0, t=40), width=820, height=620)

fig.write_html('a3_KA_H_orbits.html')
print('Saved a3_KA_H_orbits.html')
fig.show()

# (Quotient graph del 1-esqueleto removido para evitar confusión)


## 4. The Lifting Data: Transport Elements and Twisting Elements

The complex of groups $\mathcal G_{\mathscr A}(\mathcal Y)$ requires, for each morphism
$a: u \to v$ in $\mathcal Y$, a **transport element** $h_a \in H$ and, for each composable
pair $a: u \to v$, $b: v \to w$, a **twisting element** $g_{a,b} \in G_{t(a)}$.
These satisfy the cocycle identity
$$
  h_a \cdot \widetilde{t(a)} = \widetilde{t(a)}, \qquad
  g_{a,b} = h_a h_b h_{ab}^{-1},
$$
where $\widetilde a$ denotes the canonical lift of the morphism $a$ to $\mathcal X_{\mathscr A}$.

**These values are the literal input to `_compute_pi1_BH_full()`.**
Specifically, `pi1_demo.h_a` and `pi1_demo.g_ab` (computed by `_compute_twisting_elements()`)
feed directly into the four families of B\&H relations:

| Relation | Formula | Data consumed |
|----------|---------|---------------|
| R1 (group law) | $\gamma_g \gamma_h = \gamma_{gh}$ | local stabilizers $G_v$ |
| R2 (tree identification) | $\gamma_{t(a),\,\phi_a(g)} = \gamma_{i(a),\,g}$ | $h_a$ (tree edges) |
| R3 (non-tree conjugation) | $t_a\,\gamma_g\,t_a^{-1} = \gamma_{\phi_a(g)}$ | $h_a$ (non-tree edges) |
| R4 (composability) | $\gamma_{g_{a,b}}\,t_{ab} = t_a t_b$ | $g_{a,b}$ (composable pairs) |

The resulting quotient $\Gamma_{\mathscr A} = F/\langle R1, R2, R3, R4\rangle$ is the
fundamental group of the complex of groups and sits in the exact sequence
$$
  1 \longrightarrow \pi_1(K_{\mathscr A})
    \longrightarrow \Gamma_{\mathscr A}
    \xrightarrow{\;\rho_H\;} H \longrightarrow 1.
$$

**Canonical section.**
`ParabolicArrangementPi1` selects the lexicographically minimal element of each $H$-orbit
as the canonical representative.  With this choice every canonical lift $\widetilde a$
already lands on the representative of its target orbit, so $h_a = e$ for all $a$
and therefore $g_{a,b} = e$ for all composable pairs.  This is the most natural
section: all local injections $\psi_a$ are the identity and the twisting cocycle is trivial,
so **R2 and R3 collapse to a simple amalgamation** and R4 says $t_{ab} = t_a t_b$.

**Non-canonical sections** (pedagogical illustration).
The complex of groups is defined up to isomorphism regardless of section choice:
different choices produce gauge-equivalent data.  The cells below demonstrate this
concretely by applying a deterministic twist to the orbit representatives
(vertex-twist $\to$ non-trivial $h_a$) and to the edge lifts
(edge-twist $\to$ non-trivial $g_{a,b}$), and verifying the transformation formulae.


In [ ]:
# --- Canonical transport and twisting elements ---
# pi1_demo.h_a is built during __init__ from the canonical orbit representatives.
# pi1_demo._compute_twisting_elements() builds g_{a,b} for all composable pairs in Y.

# 1. Transport elements h_a (should all be identity for the canonical section)
nontrivial_ha = [(k, h) for k, h in pi1_demo.h_a.items() if h != W.one()]
print(f'Non-trivial h_a in canonical section: {len(nontrivial_ha)}')
if not nontrivial_ha:
    print('  => All h_a = e  (canonical section is compatible with the quotient).')
    print('     => R2 collapses to: gamma_{t(a), g} = gamma_{i(a), g}  (simple amalgamation).')
    print('     => R3 collapses to: t_a gamma_g t_a^{-1} = gamma_g     (trivial conjugation).')

print()

# 2. Twisting elements g_{a,b} (should all be identity when all h_a = e)
g_ab_dict = pi1_demo._compute_twisting_elements()
nontrivial_gab = [(k, g) for k, g in g_ab_dict.items() if g != W.one()]
print(f'Composable pairs (a,b) in Y: {len(g_ab_dict)}')
print(f'Non-trivial g_{{a,b}} in canonical section: {len(nontrivial_gab)}')
if not nontrivial_gab:
    print('  => All g_{a,b} = e  (canonical lifts are functorial).')
    print('     => R4 collapses to: t_{ab} = t_a t_b  (no correction by local element).')

print()
print('Summary: with the canonical section, _compute_pi1_BH_full() receives')
print('  h_a = e for all morphisms a  (stored in pi1_demo.h_a)')
print('  g_{a,b} = e for all composable pairs (a,b)  (returned by _compute_twisting_elements)')
print()
print('These values feed into R1-R4 as described in Section 4.')
print('Run Section 5 cells to see _compute_pi1_BH_full() assemble and solve the full presentation.')


### Non-canonical section: non-trivial transport elements $h_a$

If we replace the representative $\widetilde\sigma$ of orbit $\sigma$ by
$t_\sigma \cdot \widetilde\sigma$ for some $t_\sigma \in H$, the transport elements
transform as
$$
  h'_a = t_{t(a)} \cdot h_a \cdot t_{i(a)}^{-1}.
$$
Below we apply a **deterministic vertex-twist**: for every other orbit (by sorted index),
we twist by the first non-identity element of $H$.  This produces non-trivial $h'_a$
while confirming that the resulting complex of groups is equivalent (gauge-related) to the
canonical one.


In [ ]:
H_elems = [W(h) for h in H]
non_id = [h for h in H_elems if h != W.one()]

# Vertex-twist: change orbit representatives to obtain non-trivial h_a

# Deterministic twist: for every other orbit, twist by the first non-identity element of H
twist = {}
for i, oid in enumerate(sorted(pi1_demo.reps)):
    if not non_id or i % 2 == 0:
        twist[oid] = W.one()
    else:
        twist[oid] = non_id[i % len(non_id)]

# New transport: h'_a = t(v) * h_a * t(u)^{-1}
nontriv = []
for (u, v, label), h in pi1_demo.h_a.items():
    tu = twist[u]
    tv = twist[v]
    try:
        inv_tu = tu.inverse()
    except Exception:
        inv_tu = tu**-1
    h_new = tv * h * inv_tu
    if h_new != W.one():
        nontriv.append((u, v, label, h_new))

print(f'Non-trivial h_a after vertex-twist: {len(nontriv)}')
for u, v, label, h_new in nontriv[:10]:
    print(f"  {u} -> {v} ({label}): h'_a = {word_str(h_new)}")

# Confirm the transformation formula: h'_a = t_v * h_a * t_u^{-1}
for u, v, label, h_new in nontriv[:3]:
    tu = twist[u]; tv = twist[v]
    try:
        h_expected = tv * pi1_demo.h_a.get((u,v,label), W.one()) * tu.inverse()
    except Exception:
        h_expected = tv * pi1_demo.h_a.get((u,v,label), W.one()) * tu**(-1)
    print(f'  edge {u}->{v}: h_new={word_str(h_new)}, formula gives {word_str(h_expected)}, match={h_new==h_expected}')


### Edge-twist: non-trivial twisting elements $g_{a,b}$

Even with all $h_a = e$, we can produce non-trivial $g_{a,b}$ by modifying the
*edge lifts* rather than the vertex representatives.  We assign a deterministic
edge-twist $t_{\mathrm{edge}}(a) \in H$ to each morphism $a$ and redefine
$$
  h'_a = t_{\mathrm{edge}}(a) \cdot h_a.
$$
The twisting elements then satisfy
$$
  g_{a,b} = h'_a \cdot h'_b \cdot (h'_{ab})^{-1} = t_{\mathrm{edge}}(a)\, t_{\mathrm{edge}}(b)\, t_{\mathrm{edge}}(ab)^{-1},
$$
which is generically non-trivial.  This illustrates the general situation described
in Bridson–Haefliger Ch. III.C §2.9: *$h_a$ corrects each individual arrow*,
while *$g_{a,b}$ corrects the failure of functoriality for compositions*.


In [ ]:
# Edge-twist: assign a deterministic twist to each morphism in Y
# and recompute the modified transport elements h'_a = t_edge(a) * h_a.

H_elems = [W(h) for h in H]
non_id  = [h for h in H_elems if h != W.one()]

# Assign t_edge(a) for each key (u, v, label) in pi1_demo.h_a
edge_twist = {}
for i, e in enumerate(sorted(pi1_demo.h_a)):
    if not non_id or i % 2 == 0:
        edge_twist[e] = W.one()
    else:
        edge_twist[e] = non_id[i % len(non_id)]

h_prime = {e: edge_twist.get(e, W.one()) * h for e, h in pi1_demo.h_a.items()}

# Build composable pairs in Y and compute g_{a,b} = h'_a h'_b (h'_{ab})^{-1}
# (We approximate h'_{ab} by composing the two lifts and finding the correcting element.)
edges    = list(pi1_demo.Y.edges())
out_edges = {}
for u, v, label in edges:
    out_edges.setdefault(u, []).append((v, label))

nontriv_gab = []
total_pairs = 0
for u, v, la in edges:
    for w, lb in out_edges.get(v, []):
        a = (u, v, la); b = (v, w, lb)
        if a not in edge_map or b not in edge_map:
            continue
        h_a_val = h_prime.get(a); h_b_val = h_prime.get(b)
        if h_a_val is None or h_b_val is None:
            continue
        # Compute the composed lift: apply h_b to target of b, then h_a
        _, tgt_b = edge_map[b]
        cell_composed = dga._canonize((h_b_val * tgt_b[0], tgt_b[1]))
        cell_composed = dga._canonize((h_a_val * cell_composed[0], cell_composed[1]))
        rep_w = pi1_demo.reps[w]
        # Find g_ab: the unique h in H with h * cell_composed = rep_w
        g_found = None
        for h in pi1_demo.H_elements:
            if dga._canonize((h * cell_composed[0], cell_composed[1])) == rep_w:
                g_found = h
                break
        if g_found is not None and g_found != W.one():
            nontriv_gab.append((u, v, w, la, lb, g_found, cell_composed, rep_w))
        total_pairs += 1

print(f'Total composable pairs in Y: {total_pairs}')
print(f'Non-trivial g_{{a,b}} (edge-twist): {len(nontriv_gab)}')
print()
print('Sample of non-trivial g_{a,b}:')
for u, v, w, la, lb, g, _, _ in nontriv_gab[:8]:
    print(f'  a: {u}->{v} ({la}),  b: {v}->{w} ({lb})  =>  g_{{a,b}} = {word_str(g)}')


In [ ]:
# Step-by-step explanation of one non-trivial g_{a,b} instance.
# We trace through the correction: h_a corrects each arrow; g_{a,b} corrects their composition.

if not nontriv_gab:
    print('No non-trivial g_{a,b} found; try a larger edge-twist.')
else:
    u, v, w, la, lb, g_ab, cell_composed, rep_w = nontriv_gab[0]
    a = (u, v, la); b = (v, w, lb)
    _, tgt_a = edge_map[a]
    _, tgt_b = edge_map[b]

    print('Composable pair:')
    print(f'  a : {u} --> {v}  (morphism {la})')
    print(f'  b : {v} --> {w}  (morphism {lb})')
    print()
    print('Orbit representatives (chosen canonical lifts):')
    print(f'  rep({u}) = ({word_str(pi1_demo.reps[u][0])}, {pi1_demo.reps[u][1]})')
    print(f'  rep({v}) = ({word_str(pi1_demo.reps[v][0])}, {pi1_demo.reps[v][1]})')
    print(f'  rep({w}) = ({word_str(pi1_demo.reps[w][0])}, {pi1_demo.reps[w][1]})')
    print()
    print('Canonical lifts of the morphisms (arrows in X_A):')
    print(f'  lift(a) target = ({word_str(tgt_a[0])}, {tgt_a[1]})')
    print(f'  lift(b) target = ({word_str(tgt_b[0])}, {tgt_b[1]})')
    print()
    h_a_val = h_prime[a]; h_b_val = h_prime[b]
    print('Modified transport elements (edge-twist applied):')
    print(f"  h'_a = {word_str(h_a_val)}")
    print(f"  h'_b = {word_str(h_b_val)}")
    print()
    print('Composed lift (apply h_b then h_a to target of b):')
    print(f'  h_b · tgt_b = ({word_str(dga._canonize((h_b_val*tgt_b[0], tgt_b[1]))[0])}, ...)')
    print(f'  h_a · (h_b · tgt_b) = ({word_str(cell_composed[0])}, {cell_composed[1]})')
    print()
    print('Correction:')
    print(f'  rep({w}) = ({word_str(rep_w[0])}, {rep_w[1]})')
    print(f'  g_{{a,b}} = {word_str(g_ab)}  (so that g_{{a,b}} · composed_lift = rep({w}))')
    chk = dga._canonize((g_ab * cell_composed[0], cell_composed[1]))
    print(f'  Verification: g_{{a,b}} · composed_lift = ({word_str(chk[0])}, {chk[1]}) == rep({w})? {chk == rep_w}')
    print()
    print('Summary: h_a (and h_b) map each individual arrow to its orbit representative.')
    print('         g_{a,b} corrects the failure of functoriality when arrows are composed.')


## 5. Computing $\pi_1(K_{\mathscr A})$

### Three routes to the same group

`compute_pi1()` auto-selects the algorithm; two independent verification methods are also provided.

| Method | Route | Exact sequence | Extension group | Degree |
|--------|-------|----------------|-----------------|--------|
| `compute_pi1()` (W-inv.) | **W-route** | $1 \to \pi_1 \to G_{\mathscr A} \to W \to 1$ | $G_{\mathscr A}$ (modified Coxeter) | $\lvert W\rvert$ |
| `compute_pi1()` (H-inv. only) | **Full B\&H** | $1 \to \pi_1 \to \Gamma_{\mathscr A} \to H \to 1$ | $\Gamma_{\mathscr A} = \pi_1(\mathcal G(\mathcal Y))$ via R1–R4 | $\lvert H\rvert$ |
| `pi1_demo._compute_pi1_BH_full()` | **Full B\&H (explicit)** | same as above | same | $\lvert H\rvert$ |
| `verify_with_H_route()` | **H-route** | $1 \to \pi_1 \to \Gamma_{\mathscr A} \to H \to 1$ | $\Gamma_{\mathscr A} = \rho^{-1}(H) \le G_{\mathscr A}$ (subgroup of modified Coxeter) | $\lvert H\rvert$ |

**Algorithm auto-selection.** `compute_pi1(method='auto')`:
1. **W-invariant** (this example): W-route — uses the simple modified Coxeter presentation $G_{\mathscr A}$, no R1–R4 required.  Degree $|W|$.
2. **H-invariant but not W-invariant**: Full B\&H — builds $\Gamma_{\mathscr A}$ from local groups, $h_a$, $g_{a,b}$ with relations R1–R4.  Degree $|H|$.
3. **Fallback**: CW 2-skeleton route (always valid).

**Explicit B\&H as third verification.** For this W-invariant example we also run
`_compute_pi1_BH_full()` directly to confirm that the R1–R4 presentation computes
the same $\pi_1(K_{\mathscr A})$ even when the shortcut W-route would suffice.
This provides a three-way independent check.

### The modified Coxeter group $G_{\mathscr A}$ for this example


In [ ]:
# Print the modified Coxeter group G_A explicitly.
# verbose=True shows which rank-2 strata are removed and the resulting relations.
G_A, idx_map = pi1_demo.orbifold_group_presentation(verbose=True)
print()
print('G_A presentation:')
print('  Generators:', list(G_A.generators()))
print('  Relations :')
for r in G_A.relations():
    print('   ', r, '= 1')


In [ ]:
# compute_pi1() auto-selects the algorithm based on is_W_invariant().
# For this W-invariant example it uses the W-route:
#   1 -> pi_1(K_A) -> G_A -> W -> 1,  ker(rho: G_A -> Sym(W)), degree |W| = 24.
try:
    K_fp, phi_gap = pi1_demo.compute_pi1(verbose=True)
    print()
    print('pi_1(K_A) as a GAP FpGroup:', K_fp)
    ab = list(K_fp.AbelianInvariants())
    print('H_1(K_A; Z):', ab)
    print('  (0 = free Z-summand; n > 0 = Z/nZ)')
except Exception as e:
    print('Error:', e)

---
### Route 2: Full B&H complex-of-groups (explicit R1–R4 presentation)

Even though this example is $W$-invariant (so the W-route above suffices),
we run `_compute_pi1_BH_full()` directly as an **independent third verification**.
This exercises the complete Bridson–Haefliger machinery:
1. Build the spanning tree of $\mathcal Y$.
2. Index generators $\{\gamma_{v,g}\} \cup \{t_a\}$ from local stabilizers and non-tree edges.
3. Assemble relations R1–R4 using `pi1_demo.h_a` and `pi1_demo.g_ab` (all $= e$ here).
4. Map $\Gamma_{\mathscr A} \to \operatorname{Sym}(H)$ via the left-regular representation.
5. Return $\ker(\rho_H) \cong \pi_1(K_{\mathscr A})$.

The abelian invariants must match those from the W-route.


In [ ]:
# Full B&H route (explicit R1-R4 presentation), degree |H| = 6.
# This runs the complete Bridson-Haefliger machinery regardless of W-invariance.
# For W-invariant arrangements it must agree with compute_pi1() (W-route).
try:
    K_fp_BH, phi_BH = pi1_demo._compute_pi1_BH_full(verbose=True)
    ab_BH = list(K_fp_BH.AbelianInvariants())
    print()
    print('H_1(K_A; Z) via B&H full route:', ab_BH)
    ab_W  = list(K_fp.AbelianInvariants())
    print('H_1(K_A; Z) via W-route        :', ab_W)
    print('Match:', sorted(ab_BH) == sorted(ab_W))
except Exception as e:
    print('Error:', e)
    K_fp_BH = None


---
## Verification: three independent routes give the same $\pi_1$

The Bridson–Haefliger theorem guarantees a unique $\pi_1(K_{\mathscr A})$
regardless of the route taken.  For this $W$-invariant example all three
routes are available as an end-to-end consistency check:

| Route | Method | Extension group | Exact sequence | Degree |
|-------|--------|-----------------|----------------|--------|
| **W-route** | `compute_pi1()` | $G_{\mathscr A}$ (modified Coxeter) | $1\to\pi_1\to G_{\mathscr A}\to W\to 1$ | $\lvert W\rvert=24$ |
| **Full B\&H** | `_compute_pi1_BH_full()` | $\Gamma_{\mathscr A}$ via R1–R4 | $1\to\pi_1\to\Gamma_{\mathscr A}\to H\to 1$ | $\lvert H\rvert=6$ |
| **H-route** | `verify_with_H_route()` | $\Gamma_{\mathscr A}=\rho^{-1}(H)\le G_{\mathscr A}$ | $1\to\pi_1\to\Gamma_{\mathscr A}\to H\to 1$ | $\lvert H\rvert=6$ |

**Full B\&H vs H-route.** Both target the same exact sequence $1\to\pi_1\to\Gamma_{\mathscr A}\to H\to 1$
with degree $|H|=6$, but they compute $\Gamma_{\mathscr A}$ differently:
- **Full B\&H**: builds $\Gamma_{\mathscr A}$ bottom-up from the complex of groups data $(G_v, h_a, g_{a,b})$
  via generators $\{\gamma_{v,g}\}\cup\{t_a\}$ and relations R1–R4.  Does not require W-invariance.
- **H-route**: extracts $\Gamma_{\mathscr A} = \rho^{-1}(H)$ as a subgroup of $G_{\mathscr A}$
  using `PreImage` in GAP.  Requires W-invariance (so that $G_{\mathscr A}$ exists as the modified Coxeter group).

Comparing abelian invariants $H_1(K_{\mathscr A};\mathbb Z)$ across all three routes
provides a strong structural check on every component of the implementation.


In [ ]:
# H-route: ker(rho_H: Gamma_A -> Sym(H)), degree |H| = 6.
# Genuinely different coset enumeration from compute_pi1().
try:
    Gamma_fp, K_fp_H, phi_H = pi1_demo.verify_with_H_route(verbose=True)
except Exception as e:
    print('Error:', e)
    Gamma_fp = K_fp_H = None

In [ ]:
# Three-way comparison of abelian invariants H_1(K_A; Z).
# All three routes must agree.
results = {}
try:
    results['W-route'] = (list(K_fp.AbelianInvariants()),
                          f"ker(G_A -> Sym(W)), degree |W|={pi1_demo.W.cardinality()}")
except Exception:
    pass
try:
    results['B&H full'] = (list(K_fp_BH.AbelianInvariants()),
                           f"ker(Gamma_A -> Sym(H)) via R1-R4, degree |H|={len(pi1_demo.H_elements)}")
except Exception:
    pass
try:
    results['H-route'] = (list(K_fp_H.AbelianInvariants()),
                          f"ker(Gamma_A -> Sym(H)) via PreImage, degree |H|={len(pi1_demo.H_elements)}")
except Exception:
    pass

print('H_1(K_A; Z) comparison:')
for route, (ab, desc) in results.items():
    print(f'  {route:12s}: {ab}   ({desc})')

if len(results) > 1:
    all_ab = [sorted(ab) for ab, _ in results.values()]
    match = all(a == all_ab[0] for a in all_ab)
    print()
    print(f'All routes agree: {match}')
    if match:
        print('QED: three independent computations confirm pi_1(K_A).')


---
## Summary

| Object | Description | Result |
|--------|-------------|--------|
| $K_{\mathscr{A}}$ | Truncated octahedron with 8 hexagons removed | 24 verts, 36 edges, 6 squares |
| $G_f = W_{\{1,3\}}$ | Local group of retained face | $\mathbb{Z}/2 \times \mathbb{Z}/2$ |
| $\psi_{e_1\to f}$, $\psi_{e_3\to f}$ | Inclusion morphisms | $s_i \mapsto s_i$ |
| Torsion relation | Colimit at retained face | $(s_1 s_3)^2 = 1$ |
| $\mathcal{G}_{\mathscr{A}}$ | Orbifold group (modified Coxeter) | $\langle s_i\mid s_i^2, (s_1s_3)^2 \rangle$ |
| $\pi_1(K_{\mathscr{A}})$ | Kernel of $\rho: \mathcal{G} \to W$ | $F_7$ (free group of rank 7) |
| $H = \langle s_1, s_2 \rangle$ | Subgroup for B&H | $S_3$, order 6 |
| Three-way check | W-route, full B&H, H-route | All agree: $F_7$ |

**The three independent computations confirm $\pi_1(K_{\mathscr{A}}) \cong F_7$.**

---

**Version**: 3.0 | **Updated**: March 2026